# Lesson 3 - ReAct pattern with tools using langchain
This time we will add 2 tools to our agent:
- A tool that checks if a piece of text is valid Markdown
- A tool that runs `bash -n` and checks that the bash syntax is valid

The logic of these tools can be found in `_tools.py`

In [1]:
# We first install the packages
%pip install langchain langchain-openai langchainhub langchain-community --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
# We install our envirnment variables helper library
%pip install python-dotenv --quiet

# We load the library and the environment variables
import dotenv
dotenv.load_dotenv()

Note: you may need to restart the kernel to use updated packages.


True

In [3]:
# Instead of building our prompt,
# we can reuse one of the prompts available from the Langchain Hub registry.
from langchain import hub
prompt = hub.pull("hwchase17/react")

In [4]:
# We load our helper callback to see what goes over the wire
from _callbacks import PrettyPrintCallbackHandler
callback = PrettyPrintCallbackHandler()

In [5]:
# We load the llm mode using our helper and based onthe settings
import _models
import _settings
llm=_models.get_llm(_settings.MODEL_NAME, callbacks=[callback])

In [6]:
# We load our challenge from the settings
import _settings
challenge_prompt=_settings.CHALLENGE_PROMPT

In [7]:
# We define the format of the input of our tools
# In both cases the input is a string
# We use the pydantic library to define the schema
from langchain.pydantic_v1 import BaseModel, Field
class BashValidatorInput(BaseModel):
    code: str = Field(description="The code to validate")

class MarkdownValidatorInput(BaseModel):
    code: str = Field(description="The code to validate")


In [8]:
# We define the tools that we will use
# And also specify the schema to use for the input

# Note the description of the tool
# This will be used in the prompt to describe what tools are available

import _tools
from langchain.tools import BaseTool, StructuredTool, tool
bash_validator_tool = StructuredTool.from_function(
    func=_tools.bash_valid,
    name="bash_validator",
    description="Check if the code is valid bash code",
    args_schema=BashValidatorInput,
  #  return_direct=True,
    handle_tool_eror=True
)

markdown_validator_tool = StructuredTool.from_function(
    func=_tools.markdown_valid,
    name="markdown_validator",
    description="Check if the code is valid markdown",
    args_schema=MarkdownValidatorInput,
  #  return_direct=True,
    handle_tool_eror=True
)

tools = []
tools.append(bash_validator_tool)
tools.append(markdown_validator_tool)

In [9]:
# Construct the ReAct agent
from langchain.agents import AgentExecutor, create_react_agent
# Construct the ReAct agent
agent = create_react_agent(llm, tools, prompt)

In [10]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools,
         verbose=True,return_intermediate_steps=True,
           callbacks=[callback], max_iterations=15)

In [11]:
result = agent_executor.invoke({"input": challenge_prompt})

[chain][start] - inputs {'input': 'Write me a shell script to list the files in the current directory. \nOutput the script as markdown code block'}


> Entering new AgentExecutor chain...
[chat_model][start] - prompts : Answer the following questions as best you can. You have access to the following tools:

bash_validator: bash_validator(code: str) -> bool - Check if the code is valid bash code
markdown_validator: markdown_validator(code: str) -> bool - Check if the code is valid markdown

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [bash_validator, markdown_validator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: Write me a shell script to list the f

In [12]:
# Let's inspect the results
print(result["input"])
print("==================================================")
steps = result["intermediate_steps"]
for step in steps:
    print(step)
    #print(step.get"log"))
    print("==================================================")
print(result["output"])

Write me a shell script to list the files in the current directory. 
Output the script as markdown code block
(AgentAction(tool='None', tool_input='None', log='I need to write a bash script and then format it as a markdown code block.\nAction: None\nAction Input: None'), 'None is not a valid tool, try one of [bash_validator, markdown_validator].')
(AgentAction(tool='bash_validator', tool_input='ls -l', log='I will first write the bash script and validate it using the bash_validator.\nAction: bash_validator\nAction Input: "ls -l"'), True)
(AgentAction(tool='markdown_validator', tool_input='```\\nls -l\\n```', log='The bash script is valid. Now I will format it as a markdown code block and validate it using the markdown_validator.\nAction: markdown_validator\nAction Input: "```\\nls -l\\n```"'), False)
(AgentAction(tool='None', tool_input='None', log='The markdown code block is not valid. I need to check the syntax for creating a code block in markdown.\nAction: None\nAction Input: None'